<a href="https://colab.research.google.com/github/YuvanBharathi13/DeepLearning/blob/main/04_transfer_learning_in_tensorflow_part_1_feature_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

# Knowing the data

## unzipping the file

In [ ]:
import zipfile

!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

zip_ref = zipfile.ZipFile('10_food_classes_10_percent.zip')
zip_ref.extractall()
zip_ref.close()

--2025-06-06 15:52:02--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.132.207, 74.125.201.207, 74.125.202.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.132.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip’

10_food_classes_10_ 100%[===================>] 160.74M   210MB/s    in 0.8s    

2025-06-06 15:52:03 (210 MB/s) - ‘10_food_classes_10_percent.zip’ saved [168546183/168546183]



## walking through the directory

In [ ]:
import os
for dirpath, dirnames, filenames in os.walk('10_food_classes_10_percent'):
  print(f"There are {len(dirnames)} folders and {len(filenames)} images in directory {dirpath}")

There are 2 folders and 0 images in directory 10_food_classes_10_percent
There are 10 folders and 0 images in directory 10_food_classes_10_percent/train
There are 0 folders and 75 images in directory 10_food_classes_10_percent/train/sushi
There are 0 folders and 75 images in directory 10_food_classes_10_percent/train/steak
There are 0 folders and 75 images in directory 10_food_classes_10_percent/train/ramen
There are 0 folders and 75 images in directory 10_food_classes_10_percent/train/fried_rice
There are 0 folders and 75 images in directory 10_food_classes_10_percent/train/chicken_curry
There are 0 folders and 75 images in directory 10_food_classes_10_percent/train/hamburger
There are 0 folders and 75 images in directory 10_food_classes_10_percent/train/grilled_salmon
There are 0 folders and 75 images in directory 10_food_classes_10_percent/train/chicken_wings
There are 0 folders and 75 images in directory 10_food_classes_10_percent/train/ice_cream
There are 0 folders and 75 images i

## class names

In [ ]:
import pathlib
data_dir = pathlib.Path('10_food_classes_10_percent/train')
class_names = np.array(sorted(item.name for item in data_dir.glob('*')))
class_names

array(['chicken_curry', 'chicken_wings', 'fried_rice', 'grilled_salmon',
       'hamburger', 'ice_cream', 'pizza', 'ramen', 'steak', 'sushi'],
      dtype='<U14')

# Pre-processing data

In [ ]:
train_dir = '10_food_classes_10_percent/train'
test_dir = '10_food_classes_10_percent/test'

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1/255.)
test_datagen = ImageDataGenerator(rescale=1/255.)

train_data = train_datagen.flow_from_directory(train_dir,
                                               target_size=(224,224),
                                               batch_size=32,
                                               class_mode='categorical')

test_data = test_datagen.flow_from_directory(test_dir,
                                             target_size=(224,224),
                                             batch_size=32,
                                             class_mode='categorical')

Found 750 images belonging to 10 classes.
Found 2500 images belonging to 10 classes.


# callbacks

1. Experiment tracking with TensorBoard - log the performance of multiple models and then view and compare these models in a visual way on TensorBoard (a dashboard for inspecting neural network parameters). Helpful to compare the results of different models on your data.
2. Model checkpointing - save your model as it trains so you can stop training if needed and come back to continue off where you left. Helpful if training takes a long time and can't be done in one sitting.
3. Early stopping - leave your model training for an arbitrary amount of time and have it stop training automatically when it ceases to improve. Helpful when you've got a large dataset and don't know how long training will take.

In [ ]:
import datetime

def create_tensorboard_callback(dir_name,experiment_name):
  log_dir = dir_name+'/'+experiment_name+'/'+datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
  tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
  print(f"Saving tensorboard log file to {log_dir}")
  return tensorboard_callback

# Creating models using TensorFlow hub

TensorFlow Hub is a repository of trained machine learning models ready for fine-tuning and deployable anywhere.

## comparing resnet and efficient net

In [ ]:
# make sure the links are not from kaggle as they wont be compatible with tensorflow_hub functions
resnet_url = "https://tfhub.dev/google/imagenet/resnet_v2_50/feature-vector/4"
efficient_net_url = "https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1"

In [ ]:
import tensorflow_hub as hub

In [ ]:
# a function that creates a model from the model url provided
def create_model(model_url,num_classes=10):
  feature_extractor_layer = hub.KerasLayer(model_url,
                                           trainable=False, # freeze the underlying patterns
                                           name='feature_extraction_layer',
                                           input_shape=(224,224,3)) # define the input image shape

  # Create our own model
  model = tf.keras.Sequential([
    feature_extractor_layer, # use the feature extraction layer as the base
    tf.keras.layers.Dense(num_classes, activation='softmax', name='output_layer') # create our own output layer
  ])

  return model

  # model = tf.keras.models.Sequential([
  #     hub.KerasLayer(model_url,trainable=False,input_shape=(224,224,3)), # this section will not be trainable
  #     tf.keras.layers.Dense(num_classes, activation='softmax')]) # this is the only layer that will be trained

  # return model # returning an un compiled model

### creating resnet tensorflow hub feature extractor model

In [ ]:
print(tf.keras.__version__)

3.8.0


In [ ]:
import tf_keras
def create_model(model_url, num_classes):
  feature_extractor_layer = hub.KerasLayer(model_url, trainable=False, input_shape=(224,224,3), name='feature_extraction_layer')

  model = tf_keras.Sequential()
  model.add(feature_extractor_layer)
  model.add(tf_keras.layers.Dense(num_classes, activation=tf.keras.activations.softmax, name='output_layer'))

  return model

In [ ]:
efficient_net_model = create_model(efficient_net_url,10)

In [ ]:
efficient_net_model.compile(loss='categorical_crossentropy',
                            optimizer='Adam',
                            metrics=['accuracy'])

In [ ]:
efficient_net_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 feature_extraction_layer (  (None, 1280)              4049564   
 KerasLayer)                                                     
                                                                 
 output_layer (Dense)        (None, 10)                12810     
                                                                 
Total params: 4062374 (15.50 MB)
Trainable params: 12810 (50.04 KB)
Non-trainable params: 4049564 (15.45 MB)
_________________________________________________________________
